In [1]:
using DelimitedFiles
using Plots
using LaTeXStrings

const autocorr_steps = 100;

## Want to add these params somehow

params = {"text.usetex": True,
            "font.family": "serif",
            "legend.fontsize": 10,
            "axes.labelsize": 10,
            "xtick.labelsize":10,
            "ytick.labelsize":10,
            "lines.linewidth":1,
            "patch.edgecolor": "black"
         }

plt.rcParams.update(params)
plt.style.use("seaborn-deep")

In [2]:
# Compute autocorrelation times from data

function autocorr_array(gamma, stoptime, T, seed, size)
    global autocorr_steps
    
    Mag_array = readdlm("stewart_data/L$(size)/gamma$(gamma)_stoptime$(stoptime)_T$(T)_seed$(seed)_mag.dat")    
    
    M_t_avg = 0
    M_t_var = 0
    M_t_M_ti_avg = zeros(autocorr_steps)
    M_ti_avg = zeros(autocorr_steps)
    M_ti_var = zeros(autocorr_steps)

    # Need to define these to calculate running variance
    # See https://www.johndcook.com/blog/standard_deviation/
    # for algo for calculating running variance

    M_t_var_aux = 0
    M_ti_var_aux = zeros(autocorr_steps)

    autocorr_steps = 100
    data_samples = 5000

    for t = 1:data_samples - autocorr_steps + 1

        M_t_avg += Mag_array[t]

        temp_M_t_var_aux = M_t_var_aux
        M_t_var_aux = M_t_var_aux + (Mag_array[t] - M_t_var_aux) / t

        if t != 1
            M_t_var = M_t_var + (Mag_array[t] - temp_M_t_var_aux) * (Mag_array[t] - M_t_var_aux)
            end # if

        for i = 1:autocorr_steps

            M_t_M_ti_avg[i] += Mag_array[t] * Mag_array[t+i-1]
            M_ti_avg[i] += Mag_array[t+i-1]

            temp_M_ti_var_aux = M_ti_var_aux[i]
            M_ti_var_aux[i] = M_ti_var_aux[i] + (Mag_array[t+i-1] - M_ti_var_aux[i]) / t

            if t != 1
                M_ti_var[i] = M_ti_var[i] + (
                    (Mag_array[t+i-1] - temp_M_ti_var_aux) * (Mag_array[t+i-1] - M_ti_var_aux[i]))
                end # if

            end # i
        end # t

    # Divide sum by total to get averages
    M_t_avg /= data_samples - autocorr_steps + 1
    M_ti_avg /= data_samples - autocorr_steps + 1
    M_t_M_ti_avg /= data_samples - autocorr_steps + 1

    # Need to do the same for variance
    M_t_var /= data_samples - autocorr_steps + 1
    M_ti_var /= data_samples - autocorr_steps + 1

    # Get stdevs with sqrt
    M_t_stdev = sqrt(M_t_var)
    M_ti_stdev = sqrt.(M_ti_var)
    
    return (M_t_M_ti_avg .- M_t_avg * M_ti_avg) ./ (M_t_stdev * M_ti_stdev)
end

autocorr_array (generic function with 1 method)

In [3]:
# Make plot of autocorrelation times across different system sizes

system_sizes = [63, 64, 65]
time_array = 0:autocorr_steps - 1
autocorr_arrays = zeros(length(system_sizes), autocorr_steps)

for i in 1:length(system_sizes)
    autocorr = autocorr_array(0.01, 200, 0.135335, 1234, system_sizes[i])
    autocorr_arrays[i, :] = autocorr
end

plot(time_array, autocorr_arrays[1, :], label = "L = 63")
plot!(time_array, autocorr_arrays[2, :], label = "L = 64")
plot!(time_array, autocorr_arrays[3, :], label = "L = 65")

savefig("stewart_data/plots/mag_autocorr_seed1234.png")

In [4]:
# Make plot of autocorrelation times across different system sizes

system_sizes = [63, 64, 65]
time_array = 0:autocorr_steps - 1
autocorr_arrays = zeros(length(system_sizes), autocorr_steps)

for i in 1:length(system_sizes)
    autocorr = autocorr_array(0.01, 200, 0.135335, 1111, system_sizes[i])
    autocorr_arrays[i, :] = autocorr
end

plot(time_array, autocorr_arrays[1, :], label = "L = 63")
plot!(time_array, autocorr_arrays[2, :], label = "L = 64")
plot!(time_array, autocorr_arrays[3, :], label = "L = 65")

savefig("stewart_data/plots/mag_autocorr_seed1111.png")

In [5]:
# Make plot of autocorrelation times across different system sizes

system_sizes = [63, 64, 65]
time_array = 0:autocorr_steps - 1
autocorr_arrays = zeros(length(system_sizes), autocorr_steps)

for i in 1:length(system_sizes)
    autocorr = autocorr_array(0.01, 200, 0.135335, 2222, system_sizes[i])
    autocorr_arrays[i, :] = autocorr
end

plot(time_array, autocorr_arrays[1, :], label = "L = 63")
plot!(time_array, autocorr_arrays[2, :], label = "L = 64")
plot!(time_array, autocorr_arrays[3, :], label = "L = 65")

savefig("stewart_data/plots/mag_autocorr_seed2222.png")

In [6]:
# Make plot of MC energy vs. temperature across different system sizes

system_sizes = [63, 64, 65]
time_array = 0:autocorr_steps - 1
energy_arrays = zeros(length(system_sizes), 201)

for i in 1:length(system_sizes)
    size = system_sizes[i]
    energy = readdlm("stewart_data/L$(size)/gamma0.01_stoptime200_seed1234_energy.dat")
    energy_arrays[i, :] = energy
end

temp_array = readdlm("stewart_data/L64/gamma0.01_stoptime200_temperature.dat")

plot(temp_array', energy_arrays[1, :], label = "L = 63")
plot!(temp_array', energy_arrays[2, :], label = "L = 64")
plot!(temp_array', energy_arrays[3, :], label = "L = 65")

savefig("stewart_data/plots/energy_vs_temp_seed1234.png")

In [7]:
# Make plot of MC energy vs. temperature across different system sizes

system_sizes = [63, 64, 65]
time_array = 0:autocorr_steps - 1
energy_arrays = zeros(length(system_sizes), 201)

for i in 1:length(system_sizes)
    size = system_sizes[i]
    energy = readdlm("stewart_data/L$(size)/gamma0.01_stoptime200_seed1111_energy.dat")
    energy_arrays[i, :] = energy
end

temp_array = readdlm("stewart_data/L64/gamma0.01_stoptime200_temperature.dat")

plot(temp_array', energy_arrays[1, :], label = "L = 63")
plot!(temp_array', energy_arrays[2, :], label = "L = 64")
plot!(temp_array', energy_arrays[3, :], label = "L = 65")

savefig("stewart_data/plots/energy_vs_temp_seed1111.png")

In [11]:
# Make plot of MC energy vs. temperature across different system sizes

system_sizes = [63, 64, 65]
time_array = 0:autocorr_steps - 1
energy_arrays = zeros(length(system_sizes), 201)

for i in 1:length(system_sizes)
    size = system_sizes[i]
    energy = readdlm("stewart_data/L$(size)/gamma0.01_stoptime200_seed2222_energy.dat")
    energy_arrays[i, :] = energy
end

temp_array = readdlm("stewart_data/L64/gamma0.01_stoptime200_temperature.dat")

plot(temp_array', energy_arrays[1, :], label = "L = 63")
plot!(temp_array', energy_arrays[2, :], label = "L = 64")
plot!(temp_array', energy_arrays[3, :], label = "L = 65")

savefig("stewart_data/plots/energy_vs_temp_seed2222.png")